# Ridge Regression Implementation: Complete Workflow

**Purpose:** Implement ridge regression with regularization path analysis for model optimization.

**Key Concepts:**
- L2 Regularization for preventing overfitting
- Regularization path visualization
- Hyperparameter tuning (λ selection)
- Train/test validation

---

## 1. Import Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import seaborn as sns

# Set style for better visualizations
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("Libraries imported successfully!")

## 2. Core Functions: Ridge Regression

In [ ]:
def ridge_regression(xMat, yMat, lam=0.2):
    """
    Ridge Regression with L2 Regularization
    
    Formula: w = (X^T X + λI)^-1 X^T y
    
    Parameters:
        xMat: Feature matrix (numpy matrix)
        yMat: Target vector (numpy matrix)
        lam: Regularization parameter λ
    
    Returns:
        ws: Regression coefficients
    """
    xTx = xMat.T * xMat
    denom = xTx + np.eye(np.shape(xMat)[1]) * lam
    
    if np.linalg.det(denom) == 0.0:
        print("⚠️  Matrix is singular, cannot compute inverse")
        return None
    
    ws = denom.I * (xMat.T * yMat)
    return ws


def standardize_data(xMat, yMat):
    """
    Standardize features to zero mean and unit variance
    CRITICAL: Must be done before ridge regression
    """
    inxMat = xMat.copy()
    inyMat = yMat.copy()
    
    # Standardize y
    yMean = np.mean(yMat, 0)
    inyMat = yMat - yMean
    
    # Standardize X
    inMeans = np.mean(inxMat, 0)
    inVar = np.var(inxMat, 0)
    inxMat = (inxMat - inMeans) / inVar
    
    return inxMat, inyMat


def ridge_test_multiple_lambda(xArr, yArr, num_lambda=30):
    """
    Test ridge regression with multiple λ values
    Creates complete regularization path
    
    Returns:
        wMat: Coefficient matrix (each row = one λ value)
        lambda_values: List of λ values tested
    """
    xMat = np.mat(xArr)
    yMat = np.mat(yArr).T
    
    # Standardize data
    yMean = np.mean(yMat, axis=0)
    yMat = yMat - yMean
    xMeans = np.mean(xMat, axis=0)
    xVar = np.var(xMat, axis=0)
    xMat = (xMat - xMeans) / xVar
    
    # Test exponentially spaced λ values
    wMat = np.zeros((num_lambda, np.shape(xMat)[1]))
    lambda_values = []
    
    for i in range(num_lambda):
        lam = np.exp(i - 10)
        lambda_values.append(lam)
        ws = ridge_regression(xMat, yMat, lam)
        if ws is not None:
            wMat[i, :] = ws.T
    
    return wMat, lambda_values, (xMeans, xVar, yMean)


print("✅ Ridge regression functions defined")

## 3. Performance Metrics

In [ ]:
def rss_error(yArr, yHatArr):
    """Residual Sum of Squares"""
    return ((yArr - yHatArr)**2).sum()


def calculate_metrics(y_true, y_pred):
    """
    Calculate comprehensive performance metrics
    """
    metrics = {
        'RSS': rss_error(y_true, y_pred),
        'MSE': mean_squared_error(y_true, y_pred),
        'RMSE': np.sqrt(mean_squared_error(y_true, y_pred)),
        'R²': r2_score(y_true, y_pred)
    }
    return metrics


def print_metrics(metrics, label="Model"):
    """Pretty print metrics"""
    print(f"\n📊 {label} Performance:")
    print(f"   RSS  = {metrics['RSS']:>10.2f}")
    print(f"   MSE  = {metrics['MSE']:>10.2f}")
    print(f"   RMSE = {metrics['RMSE']:>10.2f}")
    print(f"   R²   = {metrics['R²']:>10.4f}")


print("✅ Metrics functions defined")

## 4. Visualization Functions

In [ ]:
def plot_regularization_path(wMat, lambda_values, feature_names=None):
    """
    Visualize how coefficients change with regularization strength
    """
    fig, ax = plt.subplots(figsize=(12, 6))
    
    # Plot each coefficient
    for i in range(wMat.shape[1]):
        label = feature_names[i] if feature_names else f'Feature {i+1}'
        ax.plot(np.log(lambda_values), wMat[:, i], label=label, linewidth=2)
    
    ax.axhline(y=0, color='black', linestyle='--', linewidth=1, alpha=0.5)
    ax.set_xlabel('log(λ) - Regularization Strength', fontsize=13, fontweight='bold')
    ax.set_ylabel('Standardized Coefficients', fontsize=13, fontweight='bold')
    ax.set_title('Ridge Regression: Regularization Path\nCoefficient Shrinkage vs. λ', 
                 fontsize=15, fontweight='bold')
    ax.legend(loc='best', fontsize=10)
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()


def plot_lambda_selection(lambda_values, train_errors, test_errors):
    """
    Plot training and test errors vs. λ to select optimal regularization
    """
    fig, ax = plt.subplots(figsize=(10, 6))
    
    ax.plot(np.log(lambda_values), train_errors, 'b-o', label='Training Error', linewidth=2)
    ax.plot(np.log(lambda_values), test_errors, 'r-s', label='Test Error', linewidth=2)
    
    # Find optimal lambda (minimum test error)
    optimal_idx = np.argmin(test_errors)
    optimal_lambda = lambda_values[optimal_idx]
    ax.axvline(x=np.log(optimal_lambda), color='green', linestyle='--', 
               linewidth=2, label=f'Optimal λ = {optimal_lambda:.4f}')
    
    ax.set_xlabel('log(λ)', fontsize=13, fontweight='bold')
    ax.set_ylabel('Mean Squared Error', fontsize=13, fontweight='bold')
    ax.set_title('λ Selection: Training vs. Test Error', fontsize=15, fontweight='bold')
    ax.legend(fontsize=11)
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    return optimal_lambda


def plot_predictions(y_true, y_pred, title="Predictions vs. Actual"):
    """
    Scatter plot of predictions vs. actual values
    """
    fig, ax = plt.subplots(figsize=(8, 8))
    
    ax.scatter(y_true, y_pred, alpha=0.6, s=50)
    
    # Perfect prediction line
    min_val = min(y_true.min(), y_pred.min())
    max_val = max(y_true.max(), y_pred.max())
    ax.plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, label='Perfect Prediction')
    
    ax.set_xlabel('Actual Values', fontsize=13, fontweight='bold')
    ax.set_ylabel('Predicted Values', fontsize=13, fontweight='bold')
    ax.set_title(title, fontsize=15, fontweight='bold')
    ax.legend(fontsize=11)
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()


print("✅ Visualization functions defined")

## 5. Generate Synthetic Data for Testing

In [ ]:
def generate_synthetic_data(n_samples=200, n_features=10, noise_level=0.5, random_state=42):
    """
    Generate synthetic regression data with known ground truth
    
    Parameters:
        n_samples: Number of samples
        n_features: Number of features
        noise_level: Standard deviation of noise
        random_state: Random seed for reproducibility
    
    Returns:
        X: Feature matrix
        y: Target vector
        true_weights: Ground truth coefficients
    """
    np.random.seed(random_state)
    
    # Generate features
    X = np.random.randn(n_samples, n_features)
    
    # Create sparse true weights (only some features are relevant)
    true_weights = np.zeros(n_features)
    true_weights[0] = 3.0
    true_weights[1] = -2.5
    true_weights[2] = 1.5
    true_weights[4] = -1.0
    true_weights[7] = 2.0
    # Features 3, 5, 6, 8, 9 are irrelevant (weight = 0)
    
    # Generate target with noise
    y = X @ true_weights + np.random.randn(n_samples) * noise_level
    
    return X, y, true_weights


# Generate data
print("🔄 Generating synthetic data...")
X, y, true_weights = generate_synthetic_data(n_samples=200, n_features=10)

print(f"✅ Data generated:")
print(f"   Samples: {X.shape[0]}")
print(f"   Features: {X.shape[1]}")
print(f"   Target range: [{y.min():.2f}, {y.max():.2f}]")
print(f"\n   True weights (ground truth):")
print(f"   {true_weights}")

## 6. Train/Test Split

In [ ]:
# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

print("✅ Data split completed:")
print(f"   Training samples: {X_train.shape[0]}")
print(f"   Test samples: {X_test.shape[0]}")
print(f"   Train/Test ratio: {X_train.shape[0]/X_test.shape[0]:.2f}")

## 7. Ridge Regression: Regularization Path Analysis

In [ ]:
print("\n" + "="*70)
print("RIDGE REGRESSION: REGULARIZATION PATH ANALYSIS")
print("="*70)

# Test 30 different lambda values
print("\n🔄 Testing 30 different λ values...")
wMat, lambda_values, scaling_params = ridge_test_multiple_lambda(
    X_train.tolist(), y_train.tolist(), num_lambda=30
)

print(f"✅ Regularization path computed")
print(f"   λ range: {min(lambda_values):.6f} to {max(lambda_values):.2f}")

# Visualize regularization path
print("\n📊 Creating regularization path visualization...")
feature_names = [f'Feature {i+1}' for i in range(X_train.shape[1])]
plot_regularization_path(wMat, lambda_values, feature_names)

## 8. Optimal λ Selection

In [ ]:
print("\n" + "="*70)
print("OPTIMAL λ SELECTION USING TRAIN/TEST ERROR")
print("="*70)

# Calculate errors for each lambda on both train and test sets
train_errors = []
test_errors = []

xMeans, xVar, yMean = scaling_params

for i, lam in enumerate(lambda_values):
    ws = np.mat(wMat[i, :]).T
    
    # Standardize and predict on training data
    X_train_std = (np.mat(X_train) - xMeans) / xVar
    y_train_pred = X_train_std * ws
    train_error = mean_squared_error(y_train - yMean.A[0], y_train_pred.A)
    train_errors.append(train_error)
    
    # Standardize and predict on test data
    X_test_std = (np.mat(X_test) - xMeans) / xVar
    y_test_pred = X_test_std * ws
    test_error = mean_squared_error(y_test - yMean.A[0], y_test_pred.A)
    test_errors.append(test_error)

# Plot error curves and find optimal lambda
print("\n📊 Creating λ selection plot...")
optimal_lambda = plot_lambda_selection(lambda_values, train_errors, test_errors)

print(f"\n✅ Optimal λ selected: {optimal_lambda:.6f}")
optimal_idx = lambda_values.index(optimal_lambda)
print(f"   Training MSE: {train_errors[optimal_idx]:.4f}")
print(f"   Test MSE: {test_errors[optimal_idx]:.4f}")

## 9. Final Model Training with Optimal λ

In [ ]:
print("\n" + "="*70)
print("FINAL MODEL: RIDGE REGRESSION WITH OPTIMAL λ")
print("="*70)

# Train final model with optimal lambda
print(f"\n🔄 Training ridge regression with λ = {optimal_lambda:.6f}...")

xMat_train = np.mat(X_train)
yMat_train = np.mat(y_train).T

# Standardize
xMat_train_std, yMat_train_std = standardize_data(xMat_train, yMat_train)

# Train
final_weights = ridge_regression(xMat_train_std, yMat_train_std, optimal_lambda)

print("\n✅ Model trained successfully!")
print(f"\n📊 Learned Coefficients:")
print(final_weights.A.flatten())

print(f"\n📊 True Weights (ground truth):")
print(true_weights)

# Calculate correlation between learned and true weights
correlation = np.corrcoef(final_weights.A.flatten(), true_weights)[0, 1]
print(f"\n📈 Correlation between learned and true weights: {correlation:.4f}")

## 10. Model Evaluation

In [ ]:
print("\n" + "="*70)
print("MODEL EVALUATION")
print("="*70)

# Predictions on training set
y_train_pred = (xMat_train_std * final_weights).A.flatten() + yMean.A[0]
train_metrics = calculate_metrics(y_train, y_train_pred)
print_metrics(train_metrics, "Training Set")

# Predictions on test set
xMat_test = np.mat(X_test)
xMat_test_std = (xMat_test - xMeans) / xVar
y_test_pred = (xMat_test_std * final_weights).A.flatten() + yMean.A[0]
test_metrics = calculate_metrics(y_test, y_test_pred)
print_metrics(test_metrics, "Test Set")

# Check for overfitting
print("\n📊 Overfitting Analysis:")
r2_diff = train_metrics['R²'] - test_metrics['R²']
if r2_diff < 0.05:
    print("   ✅ No significant overfitting detected")
elif r2_diff < 0.15:
    print("   ⚠️  Mild overfitting detected")
else:
    print("   ❌ Significant overfitting detected")
print(f"   R² difference (train - test): {r2_diff:.4f}")

## 11. Visualization: Predictions vs. Actual

In [ ]:
print("\n📊 Creating prediction visualization...")

# Training set
plot_predictions(y_train, y_train_pred, 
                title=f"Training Set: Predictions vs. Actual\nR² = {train_metrics['R²']:.4f}")

# Test set
plot_predictions(y_test, y_test_pred, 
                title=f"Test Set: Predictions vs. Actual\nR² = {test_metrics['R²']:.4f}")

## 12. Comparison: Ridge vs. Standard Linear Regression

In [ ]:
print("\n" + "="*70)
print("COMPARISON: RIDGE vs. STANDARD LINEAR REGRESSION")
print("="*70)

# Standard linear regression (no regularization)
xTx = xMat_train_std.T * xMat_train_std
if np.linalg.det(xTx) != 0.0:
    ws_standard = xTx.I * (xMat_train_std.T * yMat_train_std)
    
    # Predictions
    y_train_pred_std = (xMat_train_std * ws_standard).A.flatten() + yMean.A[0]
    y_test_pred_std = (xMat_test_std * ws_standard).A.flatten() + yMean.A[0]
    
    # Metrics
    train_metrics_std = calculate_metrics(y_train, y_train_pred_std)
    test_metrics_std = calculate_metrics(y_test, y_test_pred_std)
    
    # Comparison table
    comparison = pd.DataFrame({
        'Method': ['Standard Linear Regression', 'Ridge Regression (λ={:.6f})'.format(optimal_lambda)],
        'Train MSE': [train_metrics_std['MSE'], train_metrics['MSE']],
        'Test MSE': [test_metrics_std['MSE'], test_metrics['MSE']],
        'Train R²': [train_metrics_std['R²'], train_metrics['R²']],
        'Test R²': [test_metrics_std['R²'], test_metrics['R²']]
    })
    
    print("\n📊 Comparison Results:")
    print(comparison.to_string(index=False))
    
    # Determine winner
    if test_metrics['MSE'] < test_metrics_std['MSE']:
        print("\n✅ Ridge regression performs better (lower test MSE)")
        improvement = (test_metrics_std['MSE'] - test_metrics['MSE']) / test_metrics_std['MSE'] * 100
        print(f"   Improvement: {improvement:.2f}% reduction in test MSE")
    else:
        print("\n⚠️  Standard regression performs better on this data")
else:
    print("\n⚠️  Cannot compute standard regression (singular matrix)")
    print("   This is why ridge regression is needed!")

## 13. Summary and Conclusions

In [ ]:
print("\n" + "="*70)
print("SUMMARY")
print("="*70)

print("\n✅ Ridge Regression Implementation Complete!")
print("\n📊 Key Findings:")
print(f"   1. Optimal λ: {optimal_lambda:.6f}")
print(f"   2. Test R²: {test_metrics['R²']:.4f}")
print(f"   3. Test RMSE: {test_metrics['RMSE']:.4f}")
print(f"   4. Coefficient-truth correlation: {correlation:.4f}")

print("\n🎓 What We Learned:")
print("   • Ridge regression shrinks coefficients to prevent overfitting")
print("   • Optimal λ balances bias-variance tradeoff")
print("   • Standardization is CRITICAL before applying regularization")
print("   • Regularization path shows how coefficients change with λ")
print("   • Cross-validation helps select best λ value")

print("\n🚀 Next Steps:")
print("   1. Try with your own dataset")
print("   2. Implement k-fold cross-validation")
print("   3. Compare with Lasso and Elastic Net")
print("   4. Add interaction terms and polynomial features")
print("   5. Explore Locally Weighted Linear Regression (LWLR)")

print("\n" + "="*70)

## 14. Export Function for Your Own Data

In [ ]:
def apply_ridge_regression(X_train, y_train, X_test, y_test, lambda_range=None):
    """
    Complete ridge regression workflow for your own data
    
    Parameters:
        X_train, y_train: Training data
        X_test, y_test: Test data
        lambda_range: Optional list of lambda values to test
    
    Returns:
        optimal_lambda: Best regularization parameter
        final_weights: Trained coefficients
        test_predictions: Predictions on test set
        metrics: Performance metrics
    """
    # Convert to lists if needed
    if not isinstance(X_train, list):
        X_train = X_train.tolist()
    if not isinstance(y_train, list):
        y_train = y_train.tolist()
    
    # Get regularization path
    wMat, lambda_values, scaling_params = ridge_test_multiple_lambda(
        X_train, y_train, num_lambda=30
    )
    
    # Find optimal lambda using test set
    xMeans, xVar, yMean = scaling_params
    test_errors = []
    
    for i, lam in enumerate(lambda_values):
        ws = np.mat(wMat[i, :]).T
        X_test_std = (np.mat(X_test) - xMeans) / xVar
        y_test_pred = X_test_std * ws
        test_error = mean_squared_error(y_test - yMean.A[0], y_test_pred.A)
        test_errors.append(test_error)
    
    optimal_idx = np.argmin(test_errors)
    optimal_lambda = lambda_values[optimal_idx]
    final_weights = np.mat(wMat[optimal_idx, :]).T
    
    # Make final predictions
    X_test_std = (np.mat(X_test) - xMeans) / xVar
    test_predictions = (X_test_std * final_weights).A.flatten() + yMean.A[0]
    
    # Calculate metrics
    metrics = calculate_metrics(y_test, test_predictions)
    
    return optimal_lambda, final_weights, test_predictions, metrics


print("\n✅ Helper function 'apply_ridge_regression' ready for your data!")
print("\nUsage:")
print("  optimal_lambda, weights, predictions, metrics = apply_ridge_regression(")
print("      X_train, y_train, X_test, y_test)")

---

## Ready for Your Own Data!

To use this with your own dataset:

```python
# Load your data
X = your_features  # numpy array or pandas DataFrame
y = your_target    # numpy array or pandas Series

# Split into train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# Apply ridge regression
optimal_lambda, weights, predictions, metrics = apply_ridge_regression(
    X_train, y_train, X_test, y_test
)

# View results
print_metrics(metrics, "Your Model")
```

**Repository:** https://github.com/enzodata3-blip/Task4  
**Based on:** https://github.com/Jack-Cherish/Machine-Learning  
**Created:** 2026-02-09